In [1]:
import os
import glob
import numpy as np
import pandas as pd
import random
import math
import gc
import cv2
from tqdm import tqdm
import time
from functools import lru_cache
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import timm
from timm.scheduler import CosineLRScheduler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
import time
import io
import os
import sys
import time
import json
import math
import glob
import datetime
import argparse
import numpy as np
from pathlib import Path
from collections import defaultdict, deque

import torch
import torchvision
import torch.distributed as dist
from torch import nn, einsum
from torch.nn import functional as F
import torch.backends.cudnn as cudnn
import torch.utils.checkpoint as checkpoint

from typing import Iterable, Optional
from timm.models import create_model
from timm.optim import create_optimizer
from timm.scheduler import create_scheduler
from timm.data import Mixup,create_transform
from timm.models.registry import register_model
from timm.models.layers import DropPath, trunc_normal_
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.utils import accuracy, ModelEma,NativeScaler, get_state_dict, ModelEma

from torchvision import datasets, transforms
from torchvision.datasets.folder import ImageFolder, default_loader

from functools import partial
from einops import rearrange

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
class basNet(nn.Module):
    """
    only two viwe
    """
    def load_pretrain(self, ):
        return

    def __init__(self,):
        super(basNet, self).__init__()
        # self.output_type = ['inference', 'loss']


        self.backbone =timm.create_model ('efficientnetv2_m',
                                          pretrained=False, 
                                          drop_rate = 0.2, 
                                          drop_path_rate = 0.1,
                                          num_classes=512
                                         )

        # dim = 1280

        # self.lc  = LocalCoccurrence(dim)
        # self.gl  = GlobalConsistency(dim)
        self.mlp = nn.Sequential(
            nn.LayerNorm(1024),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Linear(512, 128),
        )#<todo> mlp needs to be deep if backbone is strong?
        self.cancer = nn.Linear(128,1)

    def forward(self, x):
        # x = batch['image']
        batch_size,C,H,W = x.shape
        x = x.reshape(-1, C, H, W)
        x_m =torch.tensor( np.array_split(batch,2,axis=3)[0])

        # print(x_m.shape)
        x_a = torch.tensor( np.array_split(batch,2,axis=3)[1])
        x_m = self.backbone(x_m)
        x_a = self.backbone(x_a)
        last = torch.cat([x_m, x_a ],-1)
        last = self.mlp(last)
        cancer = self.cancer(last).reshape(-1)

        return torch.sigmoid(cancer)

In [4]:
batch = np.random.random(size=(1,3,1024,1024))
batch = torch.tensor( batch).to(device).to(torch.float32)
batch_size,C,H,W = batch.shape
x = batch.reshape(-1, C, H, W)

In [5]:
x.shape

torch.Size([1, 3, 1024, 1024])

In [6]:
model = basNet().to(device)

In [7]:
x = model(x)
x.shape

/tmp/ipykernel_29220/2473585480.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_m =torch.tensor( np.array_split(batch,2,axis=3)[0])
/tmp/ipykernel_29220/2473585480.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_a = torch.tensor( np.array_split(batch,2,axis=3)[1])


torch.Size([1])